In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, UpSampling2D
from tensorflow.keras.layers import BatchNormalization, LeakyReLU, Concatenate
import os

2023-04-04 19:42:15.305196: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-04 19:42:15.936068: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/nickotronz7/miniconda3/envs/tf/lib/:/home/nickotronz7/miniconda3/envs/tf/lib/
2023-04-04 19:42:15.938487: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/nickotronz7/miniconda3/envs/tf/lib/:/h

In [2]:
# input_shape = (256, 256, 3)
HEIGHT = 256
WIDTH = 256
CHANNELS = 3
LR = 1e-4
EPOCHS = 10
BATCH_SIZE = 16
KernelSize = (3, 3)
#     0   1   2   3   4
f = [16, 32, 64, 128, 256]

In [3]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    "/mnt/s/Proyects/dataset/train/outdoor",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="training"
)

val_data = tf.keras.preprocessing.image_dataset_from_directory(
    "/mnt/s/Proyects/dataset/val",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="validation"
)

In [ ]:
inputs = Input(shape=(HEIGHT, WIDTH, CHANNELS))

x = inputs

# Encoder
# f[0]
d = Conv2D(f[0], KernelSize, padding='same', strides=1, activation='relu')(x)
x = BatchNormalization()(d)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2D(f[0], KernelSize, padding='same', strides=1, activation='relu')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x += d

p1 = MaxPool2D((2, 2), (2, 2))(x)
c1 = x

# f[1]
d = Conv2D(f[1], KernelSize, padding='same', strides=1, activation='relu')(p1)
x = BatchNormalization()(d)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2D(f[1], KernelSize, padding='same', strides=1, activation='relu')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x += d

p2 = MaxPool2D((2, 2), (2, 2))(x)
c2 = x

# f[2]
d = Conv2D(f[2], KernelSize, padding='same', strides=1, activation='relu')(p2)
x = BatchNormalization()(d)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2D(f[2], KernelSize, padding='same', strides=1, activation='relu')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x += d

p3 = MaxPool2D((2, 2), (2, 2))(x)
c3 = x

# Bottelneck
x = Conv2D(f[3], KernelSize, padding='same', strides=1, activation='relu')(p3)
x = LeakyReLU(alpha=0.2)(x)
x = Conv2D(f[3], KernelSize, padding='same', strides=1, activation='relu')(x)
x = LeakyReLU(alpha=0.2)(x)

# Decoder
# f[2]
x = UpSampling2D((2,2))(x)
concat = Concatenate()([x, c3])
x = Conv2D(f[2], KernelSize, padding='same', strides=1, activation='relu')(concat)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2D(f[2], KernelSize, padding='same', strides=1, activation='relu')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

# f[1]
x = UpSampling2D((2,2))(x)
concat = Concatenate()([x, c2])
x = Conv2D(f[1], KernelSize, padding='same', strides=1, activation='relu')(concat)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2D(f[1], KernelSize, padding='same', strides=1, activation='relu')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

# f[0]
x = UpSampling2D((2,2))(x)
concat = Concatenate()([x, c1])
x = Conv2D(f[0], KernelSize, padding='same', strides=1, activation='relu')(concat)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2D(f[0], KernelSize, padding='same', strides=1, activation='relu')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2D(1,(1,1), padding='same', activation="tanh", name="outputs")(x)

outputs = x



In [ ]:
model = Model(inputs, outputs)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error')

In [ ]:
model.summary()

In [ ]:
model.fit(train_data, epochs=10, validation_data=val_data)

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, batch_size=6, dim=(768, 1024), n_channels=3, shuffle=True):
        """
        Initialization
        """
        self.data = data
        self.indices = self.data.index.tolist()
        self.dim = dim
        self.n_channels = n_channels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.min_depth = 0.1
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        if (index + 1) * self.batch_size > len(self.indices):
            self.batch_size = len(self.indices) - index * self.batch_size
        # Generate one batch of data
        # Generate indices of the batch
        index = self.indices[index * self.batch_size : (index + 1) * self.batch_size]
        # Find list of IDs
        batch = [self.indices[k] for k in index]
        x, y = self.data_generation(batch)

        return x, y

    def on_epoch_end(self):

        """
        Updates indexes after each epoch
        """
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

    def load(self, image_path, depth_map, mask):
        """Load input and target image."""

        image_ = cv2.imread(image_path)
        image_ = cv2.cvtColor(image_, cv2.COLOR_BGR2RGB)
        image_ = cv2.resize(image_, self.dim)
        image_ = tf.image.convert_image_dtype(image_, tf.float32)

        depth_map = np.load(depth_map).squeeze()

        mask = np.load(mask)
        mask = mask > 0

        max_depth = min(300, np.percentile(depth_map, 99))
        depth_map = np.clip(depth_map, self.min_depth, max_depth)

# ------------------------------------------------------------------------------

        depth_map = np.log(depth_map, where=mask)

# ------------------------------------------------------------------------------

        depth_map = np.ma.masked_where(~mask, depth_map)

# ------------------------------------------------------------------------------

        depth_map = np.clip(depth_map, 0.1, np.log(max_depth, where=depth_map>0))

# ------------------------------------------------------------------------------

        depth_map = cv2.resize(depth_map, self.dim)
        depth_map = np.expand_dims(depth_map, axis=2)
        depth_map = tf.image.convert_image_dtype(depth_map, tf.float32)

        return image_, depth_map

    def data_generation(self, batch):

        x = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, *self.dim, 1))

        for i, batch_id in enumerate(batch):
            x[i,], y[i,] = self.load(
                self.data["image"][batch_id],
                self.data["depth"][batch_id],
                self.data["mask"][batch_id],
            )

        return x, y

In [ ]:
def visualize_depth_map(samples, test=False, model=None):
    input, target = samples
    cmap = plt.cm.jet
    cmap.set_bad(color="black")

    if test:
        pred = model.predict(input)
        fig, ax = plt.subplots(6, 3, figsize=(50, 50))
        for i in range(6):
            ax[i, 0].imshow((input[i].squeeze()))
            ax[i, 1].imshow((target[i].squeeze()), cmap=cmap)
            ax[i, 2].imshow((pred[i].squeeze()), cmap=cmap)
        fig.savefig("predictions"+str(datetime.datetime.today().time()).replace(':','')+".png")

    else:
        fig, ax = plt.subplots(6, 2, figsize=(50, 50))
        for i in range(6):
            ax[i, 0].imshow((input[i].squeeze()))
            ax[i, 1].imshow((target[i].squeeze()), cmap=cmap)

In [ ]:
path = "/mnt/s/Proyects/dataset/val"

import pandas as pd
import matplotlib.pyplot as plt
import datetime

filelist = []

for root, dirs, files in os.walk(path):
    for file in files:
        filelist.append(os.path.join(root, file))

filelist.sort()
data = {
    "image": [x for x in filelist if x.endswith(".png")],
    "depth": [x for x in filelist if x.endswith("_depth.npy")],
    "mask": [x for x in filelist if x.endswith("_depth_mask.npy")],
}
val_df = pd.DataFrame(data)

val_df = val_df.sample(frac=1, random_state=42)

In [ ]:
test_loader = next(
    iter(
        DataGenerator(
            data=val_df[265:].reset_index(drop="true"), batch_size=6, dim=(HEIGHT, WIDTH)
        )
    )
)
visualize_depth_map(test_loader, test=True, model=model)

test_loader = next(
    iter(
        DataGenerator(
            data=val_df[300:].reset_index(drop="true"), batch_size=6, dim=(HEIGHT, WIDTH)
        )
    )
)
visualize_depth_map(test_loader, test=True, model=model)